## やっていること
* julia01で作成した各種dfの読み込み
* 加速度をローパスフィルタでスムージング
* base_lineにlog情報を寄せる(時間が一致しない場合は線形補間)

### TODO
* logのデータをスムージングしてjoinする

In [1]:
using DataFrames
using DataFramesMeta
using CategoricalArrays
using Pipe
using JLD2
using Dates
using LeapSeconds
using Combinatorics
using Random
using Statistics
using Profile
using ProgressMeter
# using Base.Threads
using CSV
using Glob
using Formatting
using DelimitedFiles
using TextAnalysis
using BenchmarkTools
using FFTW
ENV["COLUMNS"]=700;
ENV["ROWS"]= 100
F = Float64
S = Symbol 

Symbol

In [2]:
@load "../DataFrames/train_gnss_log.jdl2" train_logdf_dict
@load "../DataFrames/test_gnss_log.jdl2" test_logdf_dict

@load "../DataFrames/train_df.jdl2" train_df
@load "../DataFrames/test_df.jdl2" test_df

@load "../DataFrames/sample_sub_df.jdl2" sample_sub_df
@load "../DataFrames/train_derived_df.jdl2" train_derived_df
# @load "../DataFrames/test_derived_df.jdl2" test_derived_df
@load "../DataFrames/train_ground_truth_df.jdl2" train_ground_truth_df


1-element Array{Symbol,1}:
 :train_ground_truth_df

In [3]:
x = train_logdf_dict[:Raw][train_logdf_dict[:Raw].Svid .== 2, :CarrierFrequencyHz]
Set(x)

Set{Float64} with 3 elements:
  1.57542003e9
  1.17645005e9
  1.59975002e9

In [4]:
train_logdf_dict[:UncalGyro]

,collectionName,phoneName,phone,time_UTC,UncalGyroXRadPerSec,UncalGyroYRadPerSec,UncalGyroZRadPerSec,DriftXRadPerSec,DriftYRadPerSec,DriftZRadPerSec,utcTimeMillis,elapsedRealtimeNanos
,Cat…?,Cat…?,Cat…?,DateTime,Float64,Float64,Float64,Float64?,Float64?,Float64?,Int64,Int64
1,2020-05-14-US-MTV-1,Pixel4XLModded,2020-05-14-US-MTV-1_Pixel4XLModded,2020-05-14T22:10:49.462,-0.161114,-0.182247,0.0639333,missing,missing,missing,1589494249462,17324898935000
2,2020-05-14-US-MTV-1,Pixel4XLModded,2020-05-14-US-MTV-1_Pixel4XLModded,2020-05-14T22:10:49.465,-0.14042,-0.167457,0.0603943,missing,missing,missing,1589494249465,17324901345469
3,2020-05-14-US-MTV-1,Pixel4XLModded,2020-05-14-US-MTV-1_Pixel4XLModded,2020-05-14T22:10:49.467,-0.118496,-0.15635,0.0580835,missing,missing,missing,1589494249467,17324903755886
4,2020-05-14-US-MTV-1,Pixel4XLModded,2020-05-14-US-MTV-1_Pixel4XLModded,2020-05-14T22:10:49.47,-0.0941107,-0.150157,0.0557858,missing,missing,missing,1589494249470,17324906166354
5,2020-05-14-US-MTV-1,Pixel4XLModded,2020-05-14-US-MTV-1_Pixel4XLModded,2020-05-14T22:10:49.472,-0.0721645,-0.146421,0.0559171,missing,missing,missing,1589494249472,17324908576771
6,2020-05-14-US-MTV-1,Pixel4XLModded,2020-05-14-US-MTV-1_Pixel4XLModded,2020-05-14T22:10:49.474,-0.0514293,-0.142695,0.0535969,missing,missing,missing,1589494249474,17324910987240
7,2020-05-14-US-MTV-1,Pixel4XLModded,2020-05-14-US-MTV-1_Pixel4XLModded,2020-05-14T22:10:49.477,-0.0331381,-0.140196,0.0537062,missing,missing,missing,1589494249477,17324913397709
8,2020-05-14-US-MTV-1,Pixel4XLModded,2020-05-14-US-MTV-1_Pixel4XLModded,2020-05-14T22:10:49.479,-0.0160363,-0.14508,0.053806,missing,missing,missing,1589494249479,17324915808125
9,2020-05-14-US-MTV-1,Pixel4XLModded,2020-05-14-US-MTV-1_Pixel4XLModded,2020-05-14T22:10:49.482,-0.00137271,-0.153646,0.0575566,missing,missing,missing,1589494249482,17324918218594


In [5]:
gnss_section_names = ["Raw","UncalAccel", "UncalGyro", "UncalMag", "Fix", "Status", "OrientationDeg"];

## 加速度をローパスフィルタでスムージング

In [6]:
function skm_m(uncal::F, bias::Union{F,Missing})::F
    if ismissing(bias)
        return uncal
    else
        return uncal - bias
    end
end
train_logdf_dict[:UncalAccel][:, :AccelXMps2] = skm_m.(train_logdf_dict[:UncalAccel][:, :UncalAccelXMps2] ,train_logdf_dict[:UncalAccel][:, :BiasXMps2])
train_logdf_dict[:UncalAccel][:, :AccelYMps2] = skm_m.(train_logdf_dict[:UncalAccel][:, :UncalAccelYMps2] ,train_logdf_dict[:UncalAccel][:, :BiasYMps2])
train_logdf_dict[:UncalAccel][:, :AccelZMps2] = skm_m.(train_logdf_dict[:UncalAccel][:, :UncalAccelZMps2] ,train_logdf_dict[:UncalAccel][:, :BiasZMps2]);
test_logdf_dict[:UncalAccel][:, :AccelXMps2] = skm_m.(test_logdf_dict[:UncalAccel][:, :UncalAccelXMps2] ,test_logdf_dict[:UncalAccel][:, :BiasXMps2])
test_logdf_dict[:UncalAccel][:, :AccelYMps2] = skm_m.(test_logdf_dict[:UncalAccel][:, :UncalAccelYMps2] ,test_logdf_dict[:UncalAccel][:, :BiasYMps2])
test_logdf_dict[:UncalAccel][:, :AccelZMps2] = skm_m.(test_logdf_dict[:UncalAccel][:, :UncalAccelZMps2] ,test_logdf_dict[:UncalAccel][:, :BiasZMps2]);

train_logdf_dict[:UncalMag][:, :MagXMicroT] = skm_m.(train_logdf_dict[:UncalMag][:, :UncalMagXMicroT] ,train_logdf_dict[:UncalMag][:, :BiasXMicroT])
train_logdf_dict[:UncalMag][:, :MagYMicroT] = skm_m.(train_logdf_dict[:UncalMag][:, :UncalMagYMicroT] ,train_logdf_dict[:UncalMag][:, :BiasYMicroT])
train_logdf_dict[:UncalMag][:, :MagZMicroT] = skm_m.(train_logdf_dict[:UncalMag][:, :UncalMagZMicroT] ,train_logdf_dict[:UncalMag][:, :BiasZMicroT])
test_logdf_dict[:UncalMag][:, :MagXMicroT] = skm_m.(test_logdf_dict[:UncalMag][:, :UncalMagXMicroT] ,test_logdf_dict[:UncalMag][:, :BiasXMicroT])
test_logdf_dict[:UncalMag][:, :MagYMicroT] = skm_m.(test_logdf_dict[:UncalMag][:, :UncalMagYMicroT] ,test_logdf_dict[:UncalMag][:, :BiasYMicroT])
test_logdf_dict[:UncalMag][:, :MagZMicroT] = skm_m.(test_logdf_dict[:UncalMag][:, :UncalMagZMicroT] ,test_logdf_dict[:UncalMag][:, :BiasZMicroT])

train_logdf_dict[:UncalGyro][:, :GyroXRadPerSec] = train_logdf_dict[:UncalGyro][:, :UncalGyroXRadPerSec]
train_logdf_dict[:UncalGyro][:, :GyroYRadPerSec] = train_logdf_dict[:UncalGyro][:, :UncalGyroYRadPerSec]
train_logdf_dict[:UncalGyro][:, :GyroZRadPerSec] = train_logdf_dict[:UncalGyro][:, :UncalGyroZRadPerSec]
test_logdf_dict[:UncalGyro][:, :GyroXRadPerSec] = test_logdf_dict[:UncalGyro][:, :UncalGyroXRadPerSec]
test_logdf_dict[:UncalGyro][:, :GyroYRadPerSec] = test_logdf_dict[:UncalGyro][:, :UncalGyroYRadPerSec]
test_logdf_dict[:UncalGyro][:, :GyroZRadPerSec] = test_logdf_dict[:UncalGyro][:, :UncalGyroZRadPerSec];

In [7]:
function lowpass(df::DataFrame, col::S, cut_off_hz = 1.0)
    col_out = S("$(col)_lowpass")
    df[!, col_out] = similar(df[!, col], F)
    gd = groupby(df, :phone)
    for gdf in gd
        x = gdf[:, col] |> Array
        t = gdf[:, :time_UTC] |> Array
        ts = mean(Dates.value.(t[2:end] - t[1:end-1]) / 1000.0)
        n = length(x)
        fftx = fft(x)
        freq_t = fftfreq(n, 1.0/ts)
        fftx[abs.(freq_t) .> cut_off_hz]  .= complex(0.0, 0.0)
        ifft_cut = ifft(fftx)
        gdf[:, col_out] = real.(ifft_cut)
    end
end
lowpass(train_logdf_dict[:UncalAccel], :AccelXMps2)
lowpass(train_logdf_dict[:UncalAccel], :AccelYMps2)
lowpass(train_logdf_dict[:UncalAccel], :AccelZMps2)
lowpass(test_logdf_dict[:UncalAccel], :AccelXMps2)
lowpass(test_logdf_dict[:UncalAccel], :AccelYMps2)
lowpass(test_logdf_dict[:UncalAccel], :AccelZMps2)

lowpass(train_logdf_dict[:UncalMag], :MagXMicroT)
lowpass(train_logdf_dict[:UncalMag], :MagYMicroT)
lowpass(train_logdf_dict[:UncalMag], :MagZMicroT)
lowpass(test_logdf_dict[:UncalMag], :MagYMicroT)
lowpass(test_logdf_dict[:UncalMag], :MagXMicroT)
lowpass(test_logdf_dict[:UncalMag], :MagZMicroT)

lowpass(train_logdf_dict[:UncalGyro], :GyroXRadPerSec)
lowpass(train_logdf_dict[:UncalGyro], :GyroZRadPerSec)
lowpass(train_logdf_dict[:UncalGyro], :GyroYRadPerSec)
lowpass(test_logdf_dict[:UncalGyro], :GyroXRadPerSec)
lowpass(test_logdf_dict[:UncalGyro], :GyroYRadPerSec)
lowpass(test_logdf_dict[:UncalGyro], :GyroZRadPerSec)

In [8]:
function get_gdf_by_key(gd::GroupedDataFrame, key::Tuple{Any})::Union{DataFrame, Missing}
    if haskey(gd.keymap, key)
        num_gdf = gd.keymap[key]
        return gd[num_gdf]
    else
        return missing
    end
end

function linear_interpolation(time::DateTime, data_time::Array{DateTime,1}, data::Array{Float64,1})::Union{F, Missing}
    #dataはソートされていることが前提
    n = length(data_time)
    time_p =  time + Dates.Millisecond(1000)
    idx_0s = searchsortedlast(data_time, time)
    idx_1e = searchsortedfirst(data_time, time_p)
    if idx_0s == 0
        return missing  #data_arr[idx1]
    end
    if idx_1e == n+1
        return missing
    end
    total_time::Float64 = (data_time[idx_1e] - data_time[idx_0s]).value |> F
    res::Float64 = 0.0
    for i in idx_0s:idx_1e-1
        Δt::Float64 = (data_time[i+1] - data_time[i]).value |> F
        res += data[i] * (Δt / total_time)
    end
    return res
end

function interpolate(base_df::DataFrame, base_time_UTC::S, data_df::DataFrame, data_time_UTC::S, data::S, data_ip::S)
    base_df[!, data_ip] = similar(base_df[!,base_time_UTC], Union{F,Missing})
    base_gd = groupby(base_df, :phone)
    data_gd = groupby(data_df, :phone)
    @showprogress "phones..." 1 for (base_gdf, key) in zip(base_gd, keys(base_gd))
        
        n = size(base_gdf, 1)
        data_gdf = get_gdf_by_key(data_gd, (key[1],))
        if  ismissing(data_gdf)
            # show("missing", key)
            base_gdf[:, data_ip] .= missing
            continue 
        end
        time = base_gdf[:, base_time_UTC]
        data_time = data_gdf[:, data_time_UTC]
        data_ =  data_gdf[:, data]
        res = Vector{Union{F, Missing}}(missing,n)
        
        # no data -> return missing
        length(data_time) == 0 && return res
        # println("hello")
        p = sortperm(data_time)
        sorted_time = data_time[p]
        sorted_data = data_[p]
        for i in 1:n
            res[i] = linear_interpolation(time[i], sorted_time, sorted_data)
        end
        base_gdf[:, data_ip] .= res
    end
end

interpolate (generic function with 1 method)

In [9]:
function combine_logdata(base_df::DataFrame, log_dfs::Dict{Symbol, DataFrame})
    dict_keys = [:UncalAccel, :UncalGyro, :UncalMag,:OrientationDeg]
    dict_cols = Dict(
        :UncalAccel => [:AccelXMps2_lowpass, :AccelYMps2_lowpass, :AccelZMps2_lowpass],
        :UncalGyro => [:GyroXRadPerSec_lowpass, :GyroYRadPerSec_lowpass, :GyroZRadPerSec_lowpass],
        :UncalMag => [:MagXMicroT_lowpass :MagYMicroT_lowpass :MagZMicroT_lowpass],
        :OrientationDeg => [:yawDeg, :rollDeg, :pitchDeg]
    )
    #:UncalAccelXMps2, :UncalAccelYMps2, :UncalAccelZMps2,
    @showprogress 1 "dict_keys..." for dict_key in dict_keys
        
        # log_gd = groupby(log_dfs[dict_key], :phone)
        log_df = log_dfs[dict_key]
        @showprogress 1  "dict_cols..."1 for dict_col in dict_cols[dict_key]
            
            base_df[!, dict_col]  = Array{Union{Float64, Missing},1}(missing, size(base_df,1))
            interpolate(base_df, :time_UTC, log_df, :time_UTC, dict_col, dict_col)
        end
    end
    return base_df
end

function combine_ground_truth(base_df::DataFrame, ground_truth_df::DataFrame)
    cols =[:time_UTC, :phone, :lngDeg, :latDeg, :heightAboveWgs84EllipsoidM, :timeSinceFirstFixSeconds, :hDop, :vDop, :speedMps, :courseDegree]
    renamed_cols = S.(String.(cols) .* "_gt")
    renamed_cols[1] = :time_UTC
    renamed_cols[2] = :phone
    gt_df = ground_truth_df[!, Vector(cols)]
    rename!(gt_df, renamed_cols)
    base_df = leftjoin(base_df, gt_df, on = [:time_UTC, :phone])
    for renamed_col in renamed_cols[3:end]
        base_df[!, renamed_col] = F.(base_df[!, renamed_col])
    end
        
    return base_df
end

combine_ground_truth (generic function with 1 method)

In [10]:
train_df_concat = combine_logdata(train_df, train_logdf_dict)
test_df_concat = combine_logdata(test_df, test_logdf_dict)

train_df_concat = combine_ground_truth(train_df_concat, train_ground_truth_df)

dict_cols...100%|█| Time: 0:00:39██████████████████| Time: 0:00:12
dict_cols...100%|█| Time: 0:00:42██████████████████| Time: 0:00:16
dict_cols...100%|█| Time: 0:00:27██████████████████| Time: 0:00:08
dict_cols...100%|█| Time: 0:00:05████████▎         |  ETA: 0:00:37
dict_keys...100%|███████████████████████████████████████| Time: 0:01:55
dict_cols...100%|█| Time: 0:00:30██████████████████| Time: 0:00:10
dict_cols...100%|█| Time: 0:00:30██████████████████| Time: 0:00:10
dict_cols...100%|█| Time: 0:00:14██████████████████| Time: 0:00:04
dict_cols...100%|█| Time: 0:00:06██████████████████| Time: 0:00:02
dict_keys...100%|███████████████████████████████████████| Time: 0:01:22


,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,heightAboveWgs84EllipsoidM,phone,time_UTC,AccelXMps2_lowpass,AccelYMps2_lowpass,AccelZMps2_lowpass,GyroXRadPerSec_lowpass,GyroYRadPerSec_lowpass,GyroZRadPerSec_lowpass,MagXMicroT_lowpass,MagYMicroT_lowpass,MagZMicroT_lowpass,yawDeg,rollDeg,pitchDeg,lngDeg_gt,latDeg_gt,heightAboveWgs84EllipsoidM_gt,timeSinceFirstFixSeconds_gt,hDop_gt,vDop_gt,speedMps_gt,courseDegree_gt
,String,String,Int64,Float64,Float64,Float64,String,DateTime,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2020-05-14-US-MTV-1,Pixel4,1273529463442,37.4236,-122.094,-34.06,2020-05-14-US-MTV-1_Pixel4,2020-05-14T22:10:45.442,0.253113,9.72609,1.18003,-0.00176897,-0.000234653,-0.00380092,25.4987,-48.4037,-15.0755,missing,missing,missing,-122.094,37.4236,33.21,551.44,1.1,0.0,0.0,12.7
2,2020-05-14-US-MTV-1,Pixel4,1273529464442,37.4236,-122.094,-33.29,2020-05-14-US-MTV-1_Pixel4,2020-05-14T22:10:46.442,0.251979,9.72617,1.18368,-0.00185471,-0.00038461,-0.00381891,25.5113,-48.3825,-14.7993,missing,missing,missing,-122.094,37.4236,33.21,552.44,1.1,0.0,0.0,12.7
3,2020-05-14-US-MTV-1,Pixel4,1273529465442,37.4236,-122.094,-30.99,2020-05-14-US-MTV-1_Pixel4,2020-05-14T22:10:47.442,0.254571,9.7268,1.17912,-0.00170001,-0.000352103,-0.00382709,25.5751,-48.3633,-13.7807,missing,missing,missing,-122.094,37.4236,33.21,553.44,1.1,0.0,0.0,12.7
4,2020-05-14-US-MTV-1,Pixel4,1273529466442,37.4236,-122.094,-32.83,2020-05-14-US-MTV-1_Pixel4,2020-05-14T22:10:48.442,0.254673,9.72623,1.17528,-0.00177008,-0.000330674,-0.00385869,25.5011,-48.519,-13.9496,missing,missing,missing,-122.094,37.4236,33.2,554.44,1.1,0.0,0.0,12.7
5,2020-05-14-US-MTV-1,Pixel4,1273529467442,37.4236,-122.094,-34.49,2020-05-14-US-MTV-1_Pixel4,2020-05-14T22:10:49.442,0.253261,9.7262,1.1742,-0.00169177,-0.000211872,-0.00381142,25.5053,-48.2951,-14.7482,missing,missing,missing,-122.094,37.4236,33.2,555.44,1.1,0.0,0.0,12.7
6,2020-05-14-US-MTV-1,Pixel4,1273529468442,37.4236,-122.094,-33.57,2020-05-14-US-MTV-1_Pixel4,2020-05-14T22:10:50.442,0.254327,9.72698,1.17938,-0.00173189,-0.000348032,-0.00386673,25.5049,-48.4427,-14.0792,missing,missing,missing,-122.094,37.4236,33.2,556.44,1.1,0.0,0.0,12.7
7,2020-05-14-US-MTV-1,Pixel4,1273529469442,37.4236,-122.094,-33.52,2020-05-14-US-MTV-1_Pixel4,2020-05-14T22:10:51.442,0.254197,9.72649,1.1795,-0.00169306,-0.000393784,-0.00391313,25.484,-48.3737,-15.6994,missing,missing,missing,-122.094,37.4236,33.19,557.44,1.1,0.0,0.0,12.7
8,2020-05-14-US-MTV-1,Pixel4,1273529470442,37.4236,-122.094,-31.6,2020-05-14-US-MTV-1_Pixel4,2020-05-14T22:10:52.442,0.25327,9.72659,1.17679,-0.0017429,-0.000355383,-0.00386882,25.4569,-48.4859,-14.1101,missing,missing,missing,-122.094,37.4236,33.19,558.44,1.1,0.0,0.0,12.7
9,2020-05-14-US-MTV-1,Pixel4,1273529471442,37.4236,-122.094,-34.22,2020-05-14-US-MTV-1_Pixel4,2020-05-14T22:10:53.442,0.254373,9.72652,1.17643,-0.00173215,-0.000259936,-0.00379819,25.6056,-48.2346,-13.724,missing,missing,missing,-122.094,37.4236,33.19,559.44,1.1,0.0,0.0,12.7


In [11]:
@save "../DataFrames/train_df_concat_20.jdl2" train_df_concat
@save "../DataFrames/test_df_concat_20.jdl2" test_df_concat

In [12]:
train_df_concat

,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,heightAboveWgs84EllipsoidM,phone,time_UTC,AccelXMps2_lowpass,AccelYMps2_lowpass,AccelZMps2_lowpass,GyroXRadPerSec_lowpass,GyroYRadPerSec_lowpass,GyroZRadPerSec_lowpass,MagXMicroT_lowpass,MagYMicroT_lowpass,MagZMicroT_lowpass,yawDeg,rollDeg,pitchDeg,lngDeg_gt,latDeg_gt,heightAboveWgs84EllipsoidM_gt,timeSinceFirstFixSeconds_gt,hDop_gt,vDop_gt,speedMps_gt,courseDegree_gt
,String,String,Int64,Float64,Float64,Float64,String,DateTime,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2020-05-14-US-MTV-1,Pixel4,1273529463442,37.4236,-122.094,-34.06,2020-05-14-US-MTV-1_Pixel4,2020-05-14T22:10:45.442,0.253113,9.72609,1.18003,-0.00176897,-0.000234653,-0.00380092,25.4987,-48.4037,-15.0755,missing,missing,missing,-122.094,37.4236,33.21,551.44,1.1,0.0,0.0,12.7
2,2020-05-14-US-MTV-1,Pixel4,1273529464442,37.4236,-122.094,-33.29,2020-05-14-US-MTV-1_Pixel4,2020-05-14T22:10:46.442,0.251979,9.72617,1.18368,-0.00185471,-0.00038461,-0.00381891,25.5113,-48.3825,-14.7993,missing,missing,missing,-122.094,37.4236,33.21,552.44,1.1,0.0,0.0,12.7
3,2020-05-14-US-MTV-1,Pixel4,1273529465442,37.4236,-122.094,-30.99,2020-05-14-US-MTV-1_Pixel4,2020-05-14T22:10:47.442,0.254571,9.7268,1.17912,-0.00170001,-0.000352103,-0.00382709,25.5751,-48.3633,-13.7807,missing,missing,missing,-122.094,37.4236,33.21,553.44,1.1,0.0,0.0,12.7
4,2020-05-14-US-MTV-1,Pixel4,1273529466442,37.4236,-122.094,-32.83,2020-05-14-US-MTV-1_Pixel4,2020-05-14T22:10:48.442,0.254673,9.72623,1.17528,-0.00177008,-0.000330674,-0.00385869,25.5011,-48.519,-13.9496,missing,missing,missing,-122.094,37.4236,33.2,554.44,1.1,0.0,0.0,12.7
5,2020-05-14-US-MTV-1,Pixel4,1273529467442,37.4236,-122.094,-34.49,2020-05-14-US-MTV-1_Pixel4,2020-05-14T22:10:49.442,0.253261,9.7262,1.1742,-0.00169177,-0.000211872,-0.00381142,25.5053,-48.2951,-14.7482,missing,missing,missing,-122.094,37.4236,33.2,555.44,1.1,0.0,0.0,12.7
6,2020-05-14-US-MTV-1,Pixel4,1273529468442,37.4236,-122.094,-33.57,2020-05-14-US-MTV-1_Pixel4,2020-05-14T22:10:50.442,0.254327,9.72698,1.17938,-0.00173189,-0.000348032,-0.00386673,25.5049,-48.4427,-14.0792,missing,missing,missing,-122.094,37.4236,33.2,556.44,1.1,0.0,0.0,12.7
7,2020-05-14-US-MTV-1,Pixel4,1273529469442,37.4236,-122.094,-33.52,2020-05-14-US-MTV-1_Pixel4,2020-05-14T22:10:51.442,0.254197,9.72649,1.1795,-0.00169306,-0.000393784,-0.00391313,25.484,-48.3737,-15.6994,missing,missing,missing,-122.094,37.4236,33.19,557.44,1.1,0.0,0.0,12.7
8,2020-05-14-US-MTV-1,Pixel4,1273529470442,37.4236,-122.094,-31.6,2020-05-14-US-MTV-1_Pixel4,2020-05-14T22:10:52.442,0.25327,9.72659,1.17679,-0.0017429,-0.000355383,-0.00386882,25.4569,-48.4859,-14.1101,missing,missing,missing,-122.094,37.4236,33.19,558.44,1.1,0.0,0.0,12.7
9,2020-05-14-US-MTV-1,Pixel4,1273529471442,37.4236,-122.094,-34.22,2020-05-14-US-MTV-1_Pixel4,2020-05-14T22:10:53.442,0.254373,9.72652,1.17643,-0.00173215,-0.000259936,-0.00379819,25.6056,-48.2346,-13.724,missing,missing,missing,-122.094,37.4236,33.19,559.44,1.1,0.0,0.0,12.7
